# 在Pytorch中构建模型

## 一.  `torch.nn.Module` and `torch.nn.Parameter`

下面介绍下Pytorch中构建深度学习神经网络模型的工具。

 除了 `Parameter` 类，本视频中讨论的所有类都是 `torch.nn.Module` 的子类。这是一个 PyTorch 的基类，旨在封装特定于 PyTorch 模型及其组件的行为。

`torch.nn.Module` 的一个重要行为是注册参数。如果某个 `Module` 子类有学习权重，这些权重将以 `torch.nn.Parameter` 的实例表示。`Parameter` 类是 `torch.Tensor` 的一个子类，具有特殊的行为，即当它们被分配为 `Module` 的属性时，它们会被添加到该模块的参数列表中。这些参数可以通过 `Module` 类的 `parameters()` 方法访问。

作为一个简单的例子，这里有一个非常简单的模型，包含两个线性层和一个激活函数。我们将创建它的一个实例，并让它报告其参数：


In [8]:
import torch

class TinyModel(torch.nn.Module):
    
    def __init__(self):
        super(TinyModel, self).__init__()
        
        self.linear1 = torch.nn.Linear(100, 200)
        self.activation = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 10)
        self.softmax = torch.nn.Softmax()
    
    def forward(self, x):
        x = self.linear1(x)
        x = self.activation(x)
        x = self.linear2(x)
        x = self.softmax(x)
        return x

tinymodel = TinyModel()

print('Model Information:')
print(tinymodel)

print('\nJust One Layer:')
print(tinymodel.linear2)

print('\nModel Parameters')
for para in tinymodel.parameters():
    print(para)
    
print('\nLayer Parameters')
for para in tinymodel.linear2.parameters():
    print(para)

Model Information:
TinyModel(
  (linear1): Linear(in_features=100, out_features=200, bias=True)
  (activation): ReLU()
  (linear2): Linear(in_features=200, out_features=10, bias=True)
  (softmax): Softmax(dim=None)
)

Just one Layer:
Linear(in_features=200, out_features=10, bias=True)

Model Parameters
Parameter containing:
tensor([[ 0.0347,  0.0155, -0.0610,  ..., -0.0927,  0.0529,  0.0664],
        [-0.0722, -0.0812,  0.0077,  ...,  0.0350,  0.0907, -0.0966],
        [-0.0651,  0.0902,  0.0804,  ..., -0.0492,  0.0146, -0.0066],
        ...,
        [ 0.0365,  0.0983,  0.0533,  ...,  0.0116, -0.0039,  0.0182],
        [-0.0544, -0.0764, -0.0781,  ..., -0.0437, -0.0065, -0.0290],
        [-0.0277,  0.0544,  0.0426,  ...,  0.0315,  0.0461,  0.0726]],
       requires_grad=True)
Parameter containing:
tensor([-0.0668, -0.0450,  0.0211,  0.0540, -0.0978,  0.0893, -0.0048,  0.0342,
        -0.0672, -0.0296,  0.0169, -0.0863, -0.0832, -0.0590,  0.0322,  0.0914,
        -0.0702, -0.0556,  0.03

这展示了 PyTorch 模型的基本结构：有一个 `__init__()` 方法用于定义模型的层和其他组件，以及一个 `forward()` 方法用于执行计算。需要注意的是，我们可以打印模型或其任何子模块，以了解其结构。

## 二. 常见的层类型

### 2.1 线性层

神经网络中最基本的层类型是 *线性* 或 *全连接* 层。这种层中，每个输入对层的每个输出都有影响，影响程度由层的权重指定。如果模型有 *m* 个输入和 *n* 个输出，那么权重将是一个 *m × n* 的矩阵。例如：


In [14]:
lin = torch.nn.Linear(3, 2)
x = torch.rand(1, 3)
print('Input:')
print(x)

print('\n\nWeight and Bias parameters:')
for param in lin.parameters():
    print(param)
    
print('\n\nWeight:')
print(lin.weight)

print('\n\nBias:')
print(lin.bias)

y = lin(x)
print('\n\nOutput:')
print(y)

Input:
tensor([[0.8060, 0.1068, 0.1357]])


Weight and Bias parameters:
Parameter containing:
tensor([[-0.4783,  0.1633,  0.1634],
        [ 0.2393,  0.4522, -0.5212]], requires_grad=True)
Parameter containing:
tensor([ 0.5287, -0.4384], requires_grad=True)


Weight
Parameter containing:
tensor([[-0.4783,  0.1633,  0.1634],
        [ 0.2393,  0.4522, -0.5212]], requires_grad=True)


Bias
Parameter containing:
tensor([ 0.5287, -0.4384], requires_grad=True)


Output:
tensor([[ 0.1829, -0.2680]], grad_fn=<AddmmBackward0>)


如果对 `x` 进行线性层权重的矩阵乘法，并加上偏置，你将得到输出向量 `y`。

还有一个重要的特性需要注意：当我们通过 `lin.weight` 检查层的权重（参数）时，它会报告自己是一个 `Parameter`（`Parameter` 是 `Tensor` 的子类），并告诉我们它正在使用自动求导跟踪梯度。这是 `Parameter` 的默认行为，与 `Tensor` 不同。

**线性层在深度学习模型中被广泛使用。你通常会在分类器模型中看到它们的身影，通常位于末尾的一个或多个线性层会有 *n* 个输出，其中 *n* 是分类器要处理的类别数。**

### 2.2 卷积层

*卷积* 层专门处理具有高度空间相关性的数据。它们在计算机视觉中非常常见，用于检测特征的紧密组合，并将其组合成更高级别的特征。它们也出现在其他情境中，例如在自然语言处理应用中，一个单词的上下文（即序列中附近的其他单词）可以影响句子的含义。

我们在早期的视频中看到了 LeNet5 中卷积层的应用：


In [15]:
import torch.functional as F


class LeNet(torch.nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel (black & white), 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = torch.nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

让我们详细分析一下这个模型中卷积层的工作原理。首先从 `conv1` 开始：

* LeNet5 设计用于接收 1x32x32 的黑白图像。**卷积层构造函数的第一个参数是输入通道的数量。** 这里是 1。如果我们要构建一个处理三色通道的模型，这个值将是 3。
* 卷积层类似于一个在图像上扫描的窗口，寻找它能识别的模式。这些模式称为*特征*，卷积层的一个参数是我们希望它学习的特征数量。**构造函数的第二个参数是输出特征的数量。** 在这里，我们要求我们的层学习 6 个特征。
* 我刚才把卷积层比作一个窗口 - 但这个窗口有多大呢？**第三个参数是窗口或*核*的大小。** 这里的“5”意味着我们选择了一个 5x5 的核。（如果你想要一个高度不同于宽度的核，可以为此参数指定一个元组 - 例如，`(3, 5)` 表示一个 3x5 的卷积核。）

卷积层的输出是一个*激活图* - 输入张量中特征存在的空间表示。`conv1` 将给我们一个 6x28x28 的输出张量；6 是特征的数量，28 是激活图的高度和宽度。（28 来自于当在 32 像素行上扫描一个 5 像素窗口时，只有 28 个有效位置。）

然后我们将卷积层的输出通过一个 ReLU 激活函数（稍后会详细介绍激活函数），然后通过一个最大池化层。最大池化层将激活图中彼此相邻的特征进行分组。它通过减少张量，将输出中的每个 2x2 组单元合并为一个单元，并将该单元的值设为进入它的 4 个单元中的最大值。这给了我们一个低分辨率的激活图，维度为 6x14x14。

我们的下一个卷积层 `conv2` 期望 6 个输入通道（对应于第一层中寻找的 6 个特征），有 16 个输出通道，并且有一个 3x3 的核。它输出一个 16x12x12 的激活图，再次通过一个最大池化层缩小为 16x6x6。在将此输出传递给线性层之前，它被重塑为一个 16 * 6 * 6 = 576 元素的向量，以供下一层使用。

有针对 1D、2D 和 3D 张量的卷积层。卷积层构造函数还有许多其他可选参数，包括步长（例如，只扫描每隔一个或每隔两个位置）、填充（这样你可以扫描到输入的边缘）等。请参阅 [文档](https://pytorch.org/docs/stable/nn.html#convolution-layers) 了解更多信息。

![卷积层示意图](https://www.baeldung.com/wp-content/uploads/sites/4/2021/10/conv_pooling-2048x677.png)


### 2.3 循环层

*循环神经网络*（或 *RNNs*）用于处理序列数据——从科学仪器的时间序列测量到自然语言句子再到 DNA 核苷酸。RNN 通过维护一个*隐藏状态*来实现，这个隐藏状态相当于对序列中迄今为止所见内容的一种记忆。

RNN 层的内部结构——或其变体 LSTM（长短期记忆）和 GRU（门控循环单元）——相对复杂，超出了本视频的范围，但我们将向你展示一个基于 LSTM 的词性标注器的实际效果（这是一种分类器，用于告诉你一个词是名词、动词等）：


In [16]:
class LSTMTagger(torch.nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = torch.nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = torch.nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = torch.nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores      
            

构造函数有四个参数：

* `vocab_size` 是输入词汇表中的单词数量。每个单词在一个 `vocab_size` 维空间中表示为一个独热向量（或单位向量）。
* `tagset_size` 是输出标签集中的标签数量。
* `embedding_dim` 是词汇表的*嵌入*空间的大小。嵌入将词汇映射到一个低维空间，在这个空间中，具有相似含义的单词彼此接近。
* `hidden_dim` 是 LSTM 的记忆单元的大小。

输入将是一个句子，单词用独热向量的索引表示。嵌入层会将这些索引映射到一个 `embedding_dim` 维的空间。LSTM 接受这个嵌入序列并迭代处理，生成一个长度为 `hidden_dim` 的输出向量。最终的线性层充当分类器；对最终层的输出应用 `log_softmax()` 将输出转换为一组归一化的概率估计，这些概率表示给定单词对应给定标签的可能性。

如果你想看到这个网络的实际运行效果，请查看 pytorch.org 上的[序列模型和 LSTM 网络](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)教程。

![LSTM](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*laH0_xXEkFE0lKJu54gkFQ.png)


### 2.3 Transformers

*Transformers* 是多功能网络，已通过像 BERT 这样的模型在 NLP 中占据了最前沿的地位。对 transformer 架构的讨论超出了本视频的范围，但 PyTorch 有一个 `Transformer` 类，可以让你定义 transformer 模型的总体参数——注意力头的数量、编码器和解码器层的数量、dropout 和激活函数等。（你甚至可以从这个单类中构建 BERT 模型，只需设置正确的参数！）`torch.nn.Transformer` 类还有一些类，用于封装各个组件（`TransformerEncoder`、`TransformerDecoder`）和子组件（`TransformerEncoderLayer`、`TransformerDecoderLayer`）。有关详细信息，请查看 [transformer 类的文档](https://pytorch.org/docs/stable/nn.html#transformer)，以及 pytorch.org 上的相关[教程](https://pytorch.org/tutorials/beginner/transformer_tutorial.html)。

[YouTube关于transformer的教程](https://youtu.be/eMlx5fFNoYc?si=RLzkSjKRMP1EjvLs)

[Transformer模型详解（图解最完整版）知乎](https://zhuanlan.zhihu.com/p/338817680)

## 三. 其他层和函数

### 3.1 数据处理层

还有一些其他类型的层在模型中执行重要功能，但**它们本身并不参与学习过程。**

**池化层**通过组合单元来减少张量，并按着某一个规则（最大化、最小化、平均等）配给输出单元。
![池化层](https://paddlepedia.readthedocs.io/en/latest/_images/avgpooling_maxpooling.png)
例如：


In [21]:
my_tensor = torch.rand(1, 6, 6)
print(my_tensor)

maxpool_layer = torch.nn.MaxPool2d(3)
avgpool_layer = torch.nn.AvgPool2d(3)
print(maxpool_layer(my_tensor))
print(avgpool_layer(my_tensor))

tensor([[[0.8179, 0.4223, 0.1031, 0.4998, 0.9807, 0.7883],
         [0.0383, 0.0148, 0.1133, 0.5124, 0.4422, 0.5419],
         [0.7136, 0.3201, 0.8491, 0.3267, 0.7967, 0.4837],
         [0.5383, 0.3686, 0.9351, 0.9668, 0.3571, 0.8391],
         [0.3417, 0.4488, 0.4175, 0.0563, 0.9025, 0.5393],
         [0.2174, 0.7326, 0.4704, 0.1715, 0.0172, 0.0554]]])
tensor([[[0.8491, 0.9807],
         [0.9351, 0.9668]]])
tensor([[[0.3769, 0.5969],
         [0.4967, 0.4339]]])


从结果能够看到，如果使用MaxPool2d，则获得的值是窗口内最大值，如果是AvgPool2d则是窗口内的平均值

**归一化层** 在将一个层的输出传递到另一个层之前，对其进行重新中心化和归一化。对中间张量进行中心化和缩放有许多有益的效果，例如允许你使用更高的学习率而**不会导致梯度爆炸或消失**。
[常见的归一化层资料](https://siyuanblog.cn/archives/normalization#toc-head-2)

在PyTorch中，有几种常用的归一化层（Normalization Layers），主要包括以下几种：

1. **Batch Normalization (批归一化)**:
   - `torch.nn.BatchNorm1d`: 适用于1D输入，比如序列数据。
   - `torch.nn.BatchNorm2d`: 适用于2D输入，比如图像数据。
   - `torch.nn.BatchNorm3d`: 适用于3D输入，比如视频数据。

2. **Layer Normalization (层归一化)**:
   - `torch.nn.LayerNorm`: 可以用于任意维度的输入，通过指定归一化维度来实现。

3. **Weight Normalization (权重归一化)**:
   - `torch.nn.utils.weight_norm`: 通过对权重进行归一化来加速训练和提高稳定性。

4. Instance Normalization (实例归一化):
   - `torch.nn.InstanceNorm1d`: 适用于1D输入。
   - `torch.nn.InstanceNorm2d`: 适用于2D输入。
   - `torch.nn.InstanceNorm3d`: 适用于3D输入。

5. Group Normalization (组归一化):
   - `torch.nn.GroupNorm`: 适用于任意维度的输入，通过指定组数进行归一化。

6. Local Response Normalization (局部响应归一化):
   - `torch.nn.LocalResponseNorm`: 一种较早使用的归一化方法，主要用于某些特定的网络结构。


这些归一化层在训练神经网络时可以帮助加速收敛、提高训练稳定性和防止过拟合。根据具体任务的需求和数据特点，可以选择合适的归一化层来应用。
下面是一些实例：


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.utils.parametrizations import weight_norm

# 定义带有Batch Normalization的前馈神经网络
class BatchNormNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BatchNormNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.bn2 = nn.BatchNorm1d(output_dim)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        return x

# 定义带有Layer Normalization的前馈神经网络
class LayerNormNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(LayerNormNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.ln1 = nn.LayerNorm(hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.ln2 = nn.LayerNorm(output_dim)

    def forward(self, x):
        x = F.relu(self.ln1(self.fc1(x)))
        x = F.relu(self.ln2(self.fc2(x)))
        return x

# 定义带有Weight Normalization的前馈神经网络
class WeightNormNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(WeightNormNN, self).__init__()
        self.fc1 = weight_norm(nn.Linear(input_dim, hidden_dim))
        self.fc2 = weight_norm(nn.Linear(hidden_dim, output_dim))

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

# 定义一个简单的训练函数
def train(model, criterion, optimizer, data, target):
    model.train()
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    return loss.item()

# 创建一些随机数据
input_dim = 10
hidden_dim = 20
output_dim = 5
batch_size = 32

data = torch.randn(batch_size, input_dim)
target = torch.randn(batch_size, output_dim)

# 创建模型、损失函数和优化器
models = {
    'BatchNorm': BatchNormNN(input_dim, hidden_dim, output_dim),
    'LayerNorm': LayerNormNN(input_dim, hidden_dim, output_dim),
    'WeightNorm': WeightNormNN(input_dim, hidden_dim, output_dim)
}

criterion = nn.MSELoss()

optimizers = {
    'BatchNorm': optim.Adam(models['BatchNorm'].parameters(), lr=0.001),
    'LayerNorm': optim.Adam(models['LayerNorm'].parameters(), lr=0.001),
    'WeightNorm': optim.Adam(models['WeightNorm'].parameters(), lr=0.001)
}

# 训练每个模型并输出损失
num_epochs = 100
for name, model in models.items():
    print(f"Training {name} model")
    optimizer = optimizers[name]
    for epoch in range(num_epochs):
        loss = train(model, criterion, optimizer, data, target)
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss:.4f}")
    print()


Training BatchNorm model
Epoch [10/100], Loss: 1.3903
Epoch [20/100], Loss: 1.2507
Epoch [30/100], Loss: 1.1410
Epoch [40/100], Loss: 1.0537
Epoch [50/100], Loss: 0.9698
Epoch [60/100], Loss: 0.8940
Epoch [70/100], Loss: 0.8412
Epoch [80/100], Loss: 0.7981
Epoch [90/100], Loss: 0.7666
Epoch [100/100], Loss: 0.7432

Training LayerNorm model
Epoch [10/100], Loss: 1.4467
Epoch [20/100], Loss: 1.3105
Epoch [30/100], Loss: 1.2118
Epoch [40/100], Loss: 1.1394
Epoch [50/100], Loss: 1.0827
Epoch [60/100], Loss: 1.0310
Epoch [70/100], Loss: 0.9883
Epoch [80/100], Loss: 0.9480
Epoch [90/100], Loss: 0.9177
Epoch [100/100], Loss: 0.8892

Training WeightNorm model
Epoch [10/100], Loss: 0.9603
Epoch [20/100], Loss: 0.9318
Epoch [30/100], Loss: 0.9101
Epoch [40/100], Loss: 0.8906
Epoch [50/100], Loss: 0.8733
Epoch [60/100], Loss: 0.8560
Epoch [70/100], Loss: 0.8392
Epoch [80/100], Loss: 0.8231
Epoch [90/100], Loss: 0.8078
Epoch [100/100], Loss: 0.7932



**Dropout 层** 是一种促进模型中*稀疏表示*的工具——即推动模型使用更少的数据进行推理。

Dropout 层的工作原理是在训练期间随机设置输入张量的部分为零——dropout 层在推理时总是关闭的。这迫使模型在这个掩码或缩减的数据集上学习。例如：

上图中，你可以看到 dropout 对样本张量的影响。你可以使用可选参数 `p` 来设置个别权重被 dropout 的概率；如果不设置，默认值是 0.5。

### 3.2 激活函数

激活函数使深度学习成为可能。神经网络实际上是一个具有许多参数的程序，它*模拟一个数学函数*。如果我们只是重复地用层权重乘以张量，我们只能模拟*线性函数*；此外，拥有多层也没有意义，因为整个网络可以简化为一个矩阵乘法。插入*非线性*激活函数在层之间，使深度学习模型能够模拟任何函数，而不仅仅是线性函数。

`torch.nn.Module` 包含封装所有主要激活函数的对象，包括 ReLU 及其多种变体、Tanh、Hardtanh、sigmoid 等。它还包括其他函数，如 Softmax，这些函数在模型的输出阶段最有用。

### 3.3 损失函数

损失函数告诉我们模型的预测与正确答案的差距。PyTorch 包含多种损失函数，包括常见的 MSE（均方误差 = L2 范数）、交叉熵损失和负似然损失（对分类器有用）等。
